In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#!pip install transformers

'''
# download Kaggle true fake news dataset
!pip install kaggle
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/Colab Notebooks"
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset
'''

from sklearn.model_selection import train_test_split
from sklearn import model_selection                   # K fold library
from torch.utils.data import DataLoader                 
from torch.optim import AdamW
import torch
import pandas as pd
import os
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, average_precision_score, average_precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import make_scorer  
from sklearn import metrics 
import torch.nn.functional as F

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 71% 29.0M/41.0M [00:00<00:00, 85.3MB/s]
100% 41.0M/41.0M [00:00<00:00, 98.9MB/s]


In [5]:
# Check GPU resources
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('Using GPU ', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('Using CPU')

Using GPU  Tesla T4


In [6]:
# Prepare tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
print(tokenizer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [7]:
# Model configuration Settings
#NUM_LABELS = 1
BATCH_SIZE = 32
MAX_LEN = 32
EPOCHS = 20

In [9]:
# Prepare dataset
path = "/content/drive/MyDrive/Colab Notebooks/CS6120_NLP/BERT/fake-and-real-news-dataset/"
df_real = pd.read_csv(path + 'True.csv')
df_fake = pd.read_csv(path + 'Fake.csv')

# Add y_true
df_real['Category'] = 1
df_fake['Category'] = 0

# Combine true news and fake news into one single file
df = df_real.append(df_fake)

<ipython-input-9-77d98d8728ce>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_real.append(df_fake)


In [10]:
# Diminish dataset to 25% due to training time
dataset_75, dataset_25 = train_test_split(df, test_size=0.01, shuffle=True)
df = dataset_25

In [11]:
from torch.utils.data import Dataset

class NewsDataset(Dataset):
  def __init__(self, train_kf, tokenizer, max_len):
    self.train_kf = train_kf
    self.tokenizer = tokenizer
    self.max_len = max_len
    

  def __getitem__(self, index):
    # convert news docment to input ids using BERT tokenizer
    text = self.train_kf.iloc[index]["text"]
    category = self.train_kf.iloc[index]["Category"]

    tokenizer_dict = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_len)
    input_ids = tokenizer_dict['input_ids']
    attention_mask = tokenizer_dict['attention_mask'] 
    y = torch.tensor(category)

    #print(input_ids, attention_mask, y)
    #print(input_ids.shape, attention_mask.shape, y.shape)

    return input_ids, attention_mask, y

  def __len__(self):
    return len(self.train_kf)

In [12]:
def train(train_dataloader, model, device, lr=2e-5, warmup_steps=200):
    model.to(device)
    model.train()
    
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1)
    batch_loss = 0
  
    for batch in train_dataloader:
      input_ids, attention_mask, labels = tuple(item.to(device) for item in batch)
      
      # without these 3 statements, despite tensor shape [batch size, seq_len] is correctly output, 
      # at here it still in [batch size, 1, seq_len] shape, so need to reshape here
      input_ids = input_ids.squeeze(1)
      attention_mask = attention_mask.squeeze(1)
      labels = labels.unsqueeze(0)

      optimizer.zero_grad() 

      # forward pass, outputs object contains the model's predictions, as well as the loss and other optional outputs.
      outputs = model(input_ids, attention_mask, labels=labels)  

      # default loss function by huggingface BERT - binary_cls: nll, multiclass: cross entropy
      # extracts the loss value from the outputs object
      loss = outputs.loss
      batch_loss += outputs.loss        # tensor in format 

      loss.backward() 
      optimizer.step() 
      scheduler.step()
    
    return batch_loss/len(train_dataloader)

In [13]:
def validate(val_dataloader, model, device):
    model.to(device)
    model.eval()

    total_loss, total_acc = 0, 0
    y_val = []
    labels_list = []

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids, attention_mask, labels = tuple(item.to(device) for item in batch)

            input_ids = input_ids.squeeze(1)
            attention_mask = attention_mask.squeeze(1)
            labels = labels.unsqueeze(0)

            # outputs produces 2 parameters: outputs.loss, outputs.logits
            outputs = model(input_ids, attention_mask, labels=labels) 

            # store prediction
            # outputs.logits represents the model's confidence scores for each class label
            y_val.append(outputs.logits.argmax(axis=1).tolist())
            labels_list.append(labels.tolist())

            #print("outputs.logits.argmax(axis=1)\n", outputs.logits.argmax(axis=1))

            y_val_flatten = [item for sublist in y_val for item in sublist]
            labels_flatten = [item for sublist1 in labels_list for sublist2 in sublist1 for item in sublist2]

    print("y_val_flatten\n", y_val_flatten)  
    print("labels_flatten\n", labels_flatten)
    print(classification_report(y_val_flatten, labels_flatten))
        

In [14]:
# Define learning rates to test
learning_rates = [0.0001, 0.001, 0.01]
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
best_lr = None
best_avg_loss = float('inf')

for fold, (train_index, valid_index) in enumerate(kf.split(df)):    # kf.split() method returns an iterator that generates a set of train and validation indices for each fold.

    # Get the training and validation data for this fold
    train_kf = df.iloc[train_index]
    val_kf = df.iloc[valid_index]

    print("fold", fold)
    print(len(train_kf))  # 5 fold is 80%
    print(len(val_kf))   # 5 fold is 20%

    # Get tokens and attention mask
    train_dataset = NewsDataset(train_kf, tokenizer, max_len=MAX_LEN)
    val_dataset = NewsDataset(val_kf, tokenizer, max_len=MAX_LEN)

    # Get dataloader
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    early_stopping_patience = 4

    for lr in learning_rates:
        print("lr", lr)

        model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
        previous_loss = 100

        for epoch in range(EPOCHS):

            # Train the model on this fold with the current learning rate
            loss = train(train_dataloader, model, device, lr=lr, warmup_steps=200)
            print("Epoch: {}, loss: {}".format(epoch, loss))

            if loss < previous_loss:
                previous_loss = loss
                num_epochs_no_improvement = 0
            else:
                num_epochs_no_improvement += 1
                if num_epochs_no_improvement == early_stopping_patience:
                    print(f"Validation loss has not improved for {early_stopping_patience} epochs. Stopping training.")
                    break

fold 0
359
90
lr 0.0001


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch: 0, loss: 0.6933373212814331
Epoch: 1, loss: 0.6306621432304382
Epoch: 2, loss: 0.5519832372665405
Epoch: 3, loss: 0.47289609909057617
Epoch: 4, loss: 0.39492639899253845
Epoch: 5, loss: 0.3263108730316162
Epoch: 6, loss: 0.26183566451072693
Epoch: 7, loss: 0.204838365316391
Epoch: 8, loss: 0.15628312528133392
Epoch: 9, loss: 0.11685194820165634
Epoch: 10, loss: 0.09029720723628998
Epoch: 11, loss: 0.06820162385702133
Epoch: 12, loss: 0.049122169613838196
Epoch: 13, loss: 0.035888563841581345
Epoch: 14, loss: 0.027322284877300262
Epoch: 15, loss: 0.019134994596242905
Epoch: 16, loss: 0.014420858584344387
Epoch: 17, loss: 0.010616103187203407
Epoch: 18, loss: 0.007773540448397398
Epoch: 19, loss: 0.005856617819517851
lr 0.001


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch: 0, loss: 0.5726157426834106
Epoch: 1, loss: 0.1076301783323288
Epoch: 2, loss: 0.017837565392255783
Epoch: 3, loss: 0.002397841541096568
Epoch: 4, loss: 0.0004013814905192703
Epoch: 5, loss: 6.212640437297523e-05
Epoch: 6, loss: 1.1269766218902078e-05
Epoch: 7, loss: 2.4452062916680006e-06
Epoch: 8, loss: 7.826212709005631e-07
Epoch: 9, loss: 3.477380801086838e-07
Epoch: 10, loss: 1.8768365350751992e-07
Epoch: 11, loss: 1.235111142250389e-07
Epoch: 12, loss: 9.069307083109379e-08
Epoch: 13, loss: 3.1354524310245324e-08
Epoch: 14, loss: 7.450580152834618e-09
Epoch: 15, loss: 1.8626450382086546e-09
Epoch: 16, loss: 6.208816794028849e-10
Epoch: 17, loss: 0.0
Epoch: 18, loss: 0.0
Epoch: 19, loss: 0.0
lr 0.01


...

Epoch: 0, loss: 0.7137813568115234
Epoch: 1, loss: 0.6369782090187073
Epoch: 2, loss: 0.592963457107544
Epoch: 3, loss: 0.5337007641792297
Epoch: 4, loss: 0.4498696029186249
Epoch: 5, loss: 0.3756522834300995
Epoch: 6, loss: 0.30392521619796753
Epoch: 7, loss: 0.24760225415229797
Epoch: 8, loss: 0.21193699538707733
Epoch: 9, loss: 0.1835031360387802
Epoch: 10, loss: 0.1507362723350525
Epoch: 11, loss: 0.1330476999282837
Epoch: 12, loss: 0.11025282740592957
Epoch: 13, loss: 0.09186389297246933
Epoch: 14, loss: 0.0761229395866394
Epoch: 15, loss: 0.05707354471087456
Epoch: 16, loss: 0.04175810515880585
Epoch: 17, loss: 0.031134789809584618
Epoch: 18, loss: 0.024029970169067383
Epoch: 19, loss: 0.017606772482395172
lr 0.001


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch: 0, loss: 0.5783482789993286
Epoch: 1, loss: 0.16038468480110168
Epoch: 2, loss: 0.015141311101615429
Epoch: 3, loss: 0.002870416734367609
Epoch: 4, loss: 0.000390561152016744
Epoch: 5, loss: 5.723785943700932e-05
Epoch: 6, loss: 1.0933961675618775e-05
Epoch: 7, loss: 2.368660716456361e-06
Epoch: 8, loss: 8.055937996687135e-07
Epoch: 9, loss: 3.501772596337105e-07
Epoch: 10, loss: 1.9340464518791123e-07
Epoch: 11, loss: 1.260389836943432e-07
Epoch: 12, loss: 8.84756374830431e-08
Epoch: 13, loss: 4.0978189730367376e-08
Epoch: 14, loss: 4.3461718668424965e-09
Epoch: 15, loss: 1.8626450382086546e-09
Epoch: 16, loss: 0.0
Epoch: 17, loss: 6.208816794028849e-10
Epoch: 18, loss: 0.0
Epoch: 19, loss: 0.0
lr 0.01


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch: 0, loss: 0.3879358768463135
Epoch: 1, loss: 0.2394430935382843
Epoch: 2, loss: 0.039990607649087906
Epoch: 3, loss: 0.0005602133460342884
Epoch: 4, loss: 1.2749783309118357e-06
Epoch: 5, loss: 3.104408285992122e-09
Epoch: 6, loss: 0.0
Epoch: 7, loss: 0.1186998039484024
Epoch: 8, loss: 0.7502238750457764
Epoch: 9, loss: 0.10780690610408783
Epoch: 10, loss: 0.5293318033218384
Validation loss has not improved for 4 epochs. Stopping training.


In [20]:
train_df, val_df = train_test_split(df, test_size=0.25, shuffle=True)

train_dataset = NewsDataset(train_df, tokenizer, max_len=MAX_LEN)
val_dataset = NewsDataset(val_df, tokenizer, max_len=MAX_LEN)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

lr = 0.001
previous_loss = 100

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

for epoch in range(EPOCHS):

    loss = train(train_dataloader, model, device, lr=lr, warmup_steps=200)
    print("Epoch: {}, loss: {}".format(epoch, loss))

    if loss < previous_loss:
        previous_loss = loss
        num_epochs_no_improvement = 0
    else:
        num_epochs_no_improvement += 1
        if num_epochs_no_improvement == early_stopping_patience:
            print(f"Validation loss has not improved for {early_stopping_patience} epochs. Stopping training.")
            break

validate(val_dataloader, model, device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch: 0, loss: 0.5782375335693359
Epoch: 1, loss: 0.24238736927509308
Epoch: 2, loss: 0.06533688306808472
Epoch: 3, loss: 0.017122521996498108
Epoch: 4, loss: 0.003183237509801984
Epoch: 5, loss: 0.0006189445266500115
Epoch: 6, loss: 9.495987615082413e-05
Epoch: 7, loss: 1.899367998703383e-05
Epoch: 8, loss: 4.916691068501677e-06
Epoch: 9, loss: 1.6245639926637523e-06
Epoch: 10, loss: 6.553123625963053e-07
Epoch: 11, loss: 3.3798542631302553e-07
Epoch: 12, loss: 1.981177035759174e-07
Epoch: 13, loss: 1.337717776550562e-07
Epoch: 14, loss: 1.1040405212270343e-07
Epoch: 15, loss: 6.637789340402378e-08
Epoch: 16, loss: 2.2690402445846303e-08
Epoch: 17, loss: 8.127905459787144e-09
Epoch: 18, loss: 2.709301893943916e-09
Epoch: 19, loss: 1.015988182473393e-09
y_val_flatten
 [0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 

In [ ]:
#!tensorboard dev upload --logdir "/content/drive/MyDrive/Colab Notebooks/CS6120 NLP/CS6120 Project 6" --name "image captioning 6.5 epochs"
